In [ ]:
import pandas as pd

import calc_error

In [ ]:
res = calc_error.calc_error_method_1(dataset_name="Tech", file_path="results_romanian")

In [ ]:
res_df = pd.DataFrame(res)

In [ ]:
import plotly.express as px

px.bar(
    res_df,
    x="model",
    y="correct",
    title="Error of the methods for the Opinion dataset",
).show()

In [ ]:
with open("results_romanian/res_method_1_Tech.txt", "w") as f:
    f.write(str(res))

In [ ]:
res

In [ ]:
res, no_correct = calc_error.calc_error_method_2(
    dataset_name="Tech", file_path="results_romanian"
)

In [ ]:
no_correct

Number of correct sentences:

Opinion:

- 3 models: 553
- 8 models: 223

Tech:

- 3 models: 382
- 8 models: 59


In [ ]:
res

In [ ]:
with open("results_romanian/res_method_2_3_models_Tech.txt", "w") as f:
    f.write(str(res))

Change threshold


In [ ]:
res, no_correct = calc_error.calc_error_method_2(
    dataset_name="Tech",
    file_path="results_romanian",
    threshold=0.9,
)

In [ ]:
no_correct

In [ ]:
res, no_correct = calc_error.calc_error_method_2(
    dataset_name="Opinion",
    file_path="results_romanian",
    threshold=0.9,
)

In [ ]:
no_correct

Paper Method


In [ ]:
import string

from stanfordcorenlp import StanfordCoreNLP
from gensim import corpora

word_token_en = {}
word_token_cn = {}

nlpEN = StanfordCoreNLP(r"./stanford-corenlp-4.5.5")


def bow(l1, l2):
    s1 = []
    if l1 not in word_token_en.keys():
        s1 = nlpEN.word_tokenize(l1)
        word_token_en[l1] = s1
    else:
        s1 = word_token_en[l1]
    s2 = []
    if l2 not in word_token_en.keys():
        s2 = nlpEN.word_tokenize(l2)
        word_token_en[l2] = s2
    else:
        s2 = word_token_en[l2]
    punc = string.punctuation
    text1 = []
    for i in s1:
        if i in punc:  # type: ignore
            continue
        text1.append(i)
    text2 = []
    for i in s2:
        if i in punc:  # type: ignore
            continue
        text2.append(i)
    texts = [text1, text2]
    dictionary = corpora.Dictionary(texts)
    # 利用doc2bow作为词袋模型
    corpus = [dictionary.doc2bow(text) for text in texts]
    dict1 = {}
    dict2 = {}
    for i, j in corpus[0]:
        dict1[i] = j
    for i, j in corpus[1]:
        dict2[i] = j
    dis = 0
    for i in range(len(dictionary.cfs.items())):
        if i not in dict1.keys():
            continue
        l1 = dict1.get(i, -1)
        l2 = dict2.get(i, -1)
        if l2 != l1:
            if l2 > l1:
                continue
            dis = dis + abs(l1 - l2)
    return dis

Number of correct sentences:

Opinion:

- 3 models: 553
- 8 models: 223

Tech:

- 3 models: 382
- 8 models: 59


In [ ]:
import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


def calc_similarity(
    sentences: list[str],
    tokenizer: AutoTokenizer,
    model: AutoModel,
) -> float:
    encoded_input = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    cosine_scores = F.cosine_similarity(
        sentence_embeddings[0].unsqueeze(0),
        sentence_embeddings[1].unsqueeze(0),
    ).item()

    return cosine_scores

In [ ]:
import pickle
import dataclasses

import pandas as pd
from calc_error import calc_correct

file_path = "./results_romanian/"
dataset_name = "Opinion"

df_res = {}

to_analyze_res = []


@dataclasses.dataclass
class Result:
    dataset_name: str
    model: str
    threshold: int
    l1: str
    l2: str
    is_suspicious: bool
    is_wrong: bool


for dataset_name in ["Opinion", "Tech"]:
    results = []
    sentence_map = calc_correct(dataset_name, file_path)

    for threshold in [0, 2, 4, 6, 8, 10, 12]:
        for model_ in ["deepl", "google", "bing"]:
            checkpoint_name = model_.replace("/", "_").replace("-", "_")

            file_name = (
                f"./{file_path}/target_sentences_{checkpoint_name}_{dataset_name}.pkl"
            )

            with open(f"./{file_path}/datasetAfter_{dataset_name}.pkl", "rb") as f:
                dataset = pickle.load(f)

            with open(file_name, "rb") as f:
                model_sentences: dict[str, str] = pickle.load(f)

            no_total = len(model_sentences)
            no_suspicous = 0
            no_correct = 0

            for key, values in dataset.items():
                for i in range(1, len(values)):
                    is_suspicious = False
                    is_wrong = False

                    l1 = model_sentences.get(values[i], "")
                    l2 = model_sentences.get(values[i - 1], "")
                    if len(l1) > len(l2):
                        continue
                    res = bow(l1, l2)
                    if res > len(l1.split(" ")):
                        continue
                    if res > threshold:
                        no_suspicous += 1
                        is_suspicious = True
                        if sentence_map[values[i]] is False:
                            no_correct += 1
                            is_wrong = True

                    to_analyze_res.append(
                        Result(
                            dataset_name=dataset_name,
                            model=model_,
                            threshold=threshold,
                            l1=l1,
                            l2=l2,
                            is_suspicious=is_suspicious,
                            is_wrong=is_wrong,
                        )
                    )

            no_wrong = 0
            if dataset_name == "Opinion":
                no_wrong = no_total - 553
            else:
                no_wrong = no_total - 382

            results.append(
                {
                    "threshold": threshold,
                    "model": model_,
                    "correct": no_total - no_suspicous,
                    "total": no_total,
                    "suspicous": no_suspicous,
                    "precision": no_correct / no_suspicous * 100,
                    "percentage": no_suspicous / no_total * 100,
                }
            )

    df_res[dataset_name] = pd.DataFrame(results)

    df_res[dataset_name].to_csv(
        f"./results_romanian/res_method_3_{dataset_name}.csv", index=False
    )

pd.DataFrame(to_analyze_res).to_csv(
    "./results_romanian/res_method_3_to_analyze.csv", index=False
)

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

# display bar chart and line chart on the same figure
# there are three models: deepl, google, bing and three lines

for dataset_name in ["Opinion", "Tech"]:
    df = df_res[dataset_name]
    fig = make_subplots(
        specs=[[{"secondary_y": True}]],
        rows=1,
        cols=1,
    )
    for i in ["deepl", "google", "bing"]:
        fig.add_trace(
            px.bar(
                df.query(f"model == '{i}'"),
                x="threshold",
                y="suspicous",
                color="model",
                color_discrete_map={"deepl": "blue", "google": "green", "bing": "red"},
            ).data[0],
            secondary_y=False,
        )

    for i in ["deepl", "google", "bing"]:
        fig.add_trace(
            px.line(
                df.query(f"model == '{i}'"),
                x="threshold",
                y="precision",
                color="model",
                color_discrete_map={
                    "deepl": "darkblue",
                    "google": "darkgreen",
                    "bing": "darkred",
                },
            ).data[0],
            secondary_y=True,
        )

    fig.update_layout(
        title=f"Error of the methods for the {dataset_name} dataset",
        xaxis_title="Threshold",
        yaxis_title="Number of suspicious sentences",
        yaxis2_title="Precision",
    )
    fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

file_path = "./results_romanian/"
dataset_name = "Opinion"


for dataset_name in ["Opinion", "Tech"]:
    results = []
    for threshold in [0, 2, 4, 6, 8, 10, 12]:
        for model_ in [
            "deepl",
            "google",
            "bing",
            "Helsinki-NLP/opus-mt-en-ro",
            "Helsinki-NLP/opus-tatoeba-en-ro",
            "Helsinki-NLP/opus-mt-tc-big-en-ro",
            "facebook/mbart-large-en-ro",
            "BlackKakapo/opus-mt-en-ro",
        ]:
            checkpoint_name = model_.replace("/", "_").replace("-", "_")

            file_name = (
                f"./{file_path}/target_sentences_{checkpoint_name}_{dataset_name}.pkl"
            )

            with open(f"./{file_path}/datasetAfter_{dataset_name}.pkl", "rb") as f:
                dataset = pickle.load(f)

            with open(file_name, "rb") as f:
                model_sentences: dict[str, str] = pickle.load(f)

            no_total = len(model_sentences)
            no_suspicous = 0

            for key, values in dataset.items():
                if len(values) == 0:
                    continue
                res = bow(values[0], key)
                if res > threshold:
                    no_suspicous += 1

                for i in range(1, len(values)):
                    l1 = model_sentences.get(values[i], "")
                    l2 = model_sentences.get(values[i - 1], "")
                    if len(l1) > len(l2):
                        continue
                    res = bow(l1, l2)
                    if res > len(l1.split(" ")):
                        continue
                    if res > threshold:
                        no_suspicous += 1

            results.append(
                {
                    "threshold": threshold,
                    "model": model_,
                    "correct": no_total - no_suspicous,
                    "total": no_total,
                    "suspicous": no_suspicous,
                    "percentage": no_suspicous / no_total * 100,
                }
            )

    px.bar(
        pd.DataFrame(results),
        x="threshold",
        y="suspicous",
        color="model",
        barmode="group",
        title=f"Error of the methods for the {dataset_name} dataset",
    ).show()

    px.bar(
        pd.DataFrame(results),
        x="threshold",
        y="suspicous",
        color="model",
        barmode="group",
        title=f"Error of the methods for the {dataset_name} dataset (percentage)",
    ).show()

In [ ]:
import pickle
import dataclasses

import pandas as pd
from calc_error import calc_correct

file_path = "./results_romanian/"

df_res = {}

to_analyze_res = []


@dataclasses.dataclass
class Result:
    dataset_name: str
    model: str
    threshold: int
    l1: str
    l2: str
    is_suspicious: bool
    is_wrong: bool


for dataset_name in ["Opinion", "Tech"]:
    results = []
    sentence_map = calc_correct(dataset_name, file_path)

    for threshold in [0, 2, 4, 6, 8, 10, 12]:
        for model_ in [
            "deepl",
            "google",
            "bing",
            "Helsinki-NLP/opus-mt-en-ro",
            "Helsinki-NLP/opus-tatoeba-en-ro",
            "Helsinki-NLP/opus-mt-tc-big-en-ro",
            "facebook/mbart-large-en-ro",
            "BlackKakapo/opus-mt-en-ro",
        ]:
            checkpoint_name = model_.replace("/", "_").replace("-", "_")

            file_name = (
                f"./{file_path}/target_sentences_{checkpoint_name}_{dataset_name}.pkl"
            )

            with open(f"./{file_path}/datasetAfter_{dataset_name}.pkl", "rb") as f:
                dataset = pickle.load(f)

            with open(file_name, "rb") as f:
                model_sentences: dict[str, str] = pickle.load(f)

            no_total = len(model_sentences)
            no_suspicous = 0
            no_correct = 0

            for key, values in dataset.items():
                for i in range(1, len(values)):
                    is_suspicious = False
                    is_wrong = False

                    l1 = model_sentences.get(values[i], "")
                    l2 = model_sentences.get(values[i - 1], "")
                    if len(l1) > len(l2):
                        continue
                    res = bow(l1, l2)
                    if res > len(l1.split(" ")):
                        continue
                    if res > threshold:
                        no_suspicous += 1
                        is_suspicious = True
                        if sentence_map[values[i]] is False:
                            no_correct += 1
                            is_wrong = True

                    to_analyze_res.append(
                        Result(
                            dataset_name=dataset_name,
                            model=model_,
                            threshold=threshold,
                            l1=l1,
                            l2=l2,
                            is_suspicious=is_suspicious,
                            is_wrong=is_wrong,
                        )
                    )

            no_wrong = 0
            if dataset_name == "Opinion":
                no_wrong = no_total - 553
            else:
                no_wrong = no_total - 382

            results.append(
                {
                    "threshold": threshold,
                    "model": model_,
                    "correct": no_total - no_suspicous,
                    "total": no_total,
                    "suspicous": no_suspicous,
                    "precision": no_correct / no_suspicous * 100,
                    "percentage": no_suspicous / no_total * 100,
                }
            )

    df_res[dataset_name] = pd.DataFrame(results)

    df_res[dataset_name].to_csv(
        f"./results_romanian/res_method_8_{dataset_name}.csv", index=False
    )

pd.DataFrame(to_analyze_res).to_csv(
    "./results_romanian/res_method_8_to_analyze.csv", index=False
)

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

# display bar chart and line chart on the same figure
# there are three models: deepl, google, bing and three lines

models = [
    "deepl",
    "google",
    "bing",
    "Helsinki-NLP/opus-mt-en-ro",
    "Helsinki-NLP/opus-tatoeba-en-ro",
    "Helsinki-NLP/opus-mt-tc-big-en-ro",
    "facebook/mbart-large-en-ro",
    "BlackKakapo/opus-mt-en-ro",
]

color_discrete_map = {
    "deepl": "blue",
    "google": "green",
    "bing": "red",
    "Helsinki-NLP/opus-mt-en-ro": "cyan",
    "Helsinki-NLP/opus-tatoeba-en-ro": "goldenrod",
    "Helsinki-NLP/opus-mt-tc-big-en-ro": "khaki",
    "facebook/mbart-large-en-ro": "magenta",
    "BlackKakapo/opus-mt-en-ro": "salmon",
}

for dataset_name in ["Opinion", "Tech"]:
    df = df_res[dataset_name]
    fig = make_subplots(
        specs=[[{"secondary_y": True}]],
        rows=1,
        cols=1,
    )
    for i in models:
        fig.add_trace(
            px.bar(
                df.query(f"model == '{i}'"),
                x="threshold",
                y="suspicous",
                color="model",
                color_discrete_map=color_discrete_map,
            ).data[0],
            secondary_y=False,
        )

    for i in models:
        fig.add_trace(
            px.line(
                df.query(f"model == '{i}'"),
                x="threshold",
                y="precision",
                color="model",
                color_discrete_map={
                    key: f"dark{value}" for key, value in color_discrete_map.items()
                },
            ).data[0],
            secondary_y=True,
        )

    fig.update_layout(
        title=f"Error of the methods for the {dataset_name} dataset",
        xaxis_title="Threshold",
        yaxis_title="Number of suspicious sentences",
        yaxis2_title="Precision",
        height=600,
    )
    fig.show()